In [2]:
import torch
import torch.nn as nn
import numpy as np
import gym

In [5]:
env = gym.make('CartPole-v1')
env.unwrapped

In [10]:
state_space = env.observation_space.shape[0]
action_space = env.action_space.n

In [11]:
class Policynetwork(nn.Module):
    def __init__(self, state_space, action_space):
        super(Policynetwork, self).__init__()
        
        self.linear1 = nn.Linear(state_space, 128)
        self.linear2 = nn.Linear(128, 64)
        self.linear3 = nn.Linear(64, action_space)
        
    def forward(self, state):
        
        hidden1 = nn.functional.relu(self.linear1(state))
        hidden2 = nn.functional.relu(self.linear2(hidden1))
        action_preds = nn.functional.softmax(self.linear(hidden2))
        
        return action_preds
    

policy_network = Policynetwork(state_space, action_space)

In [12]:
alpha = 0.003

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(policy_network.parameters(), lr = alpha)

In [14]:
def epsilon_greedy(predictions, epsilon):
    
    chance = np.random.uniform(0,1)
    return torch.argmax(predictions) if chance<epsilon else env.action_space.sample()

def generate_episode(env, policy):
    
    experience = []
    total_reward = 0
    state = env.reset()
    action = epsilon_greedy(policy.forward(torch.tensor(state)))
    
    done = False
    while not done:
        nextstate, reward, done, _ = env.step(action)
        total_reward += reward
        experience.append((state, action, reward, nextstate))
        state = nextstate
        action = epsilon_greedy(policy_network(state))
    
    return experience, total_reward

In [16]:
## TRAIN

episodes = 1000
epsilon = 0.9

# state = env.reset()
# action_preds = policy_network.forward(torch.tensor(state))

for episode in range(episodes):
    
    episode_experience, total_reward = generate_episode(env, policy_network)
    for sars in episode_experience:
        
        state, action, reward, nextstate = sars
        target = torch.tensor(total_reward)        # using monte carlo estimate for target
        

RuntimeError: Expected object of scalar type Double but got scalar type Float for argument #3 'mat2' in call to _th_addmm_out